# Catnip

In [ ]:
# Autoreload for debugging
%load_ext autoreload
%autoreload 2

In [ ]:
# --- Scripts for Colab ---
# Clone repo
!git clone -b yolo-finetune --recurse-submodules https://github.com/rifusaki/catnip.git
%cd /content/catnip
!git checkout yolo-finetune

# Authenticate with Google
from google.colab import auth
auth.authenticate_user()

# Install gcsfuse
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

# Mount bucket
!mkdir -p /content/catnip/data
!gcsfuse --implicit-dirs catnip-data /content/catnip/data

# Install packages not included in Colab
%pip install ultralytics pydantic pydantic-settings omegaconf

In [ ]:
import os
from pathlib import Path

# Move working directory to project root (one level up from /notebooks)
os.chdir(Path.cwd().parent) # Local
os.chdir(str(Path.cwd())+'/catnip') # Colab

print("Working directory set to:", Path.cwd())

Working directory set to: /Users/rifusaki/repos/catnip


In [ ]:
# Dependencies and configuration
from src.config import settings, setup_dirs

izutsumiPaths, notIzutsumiPaths = setup_dirs()

## Pre-processing

### Panel extraction

In [ ]:
from modules.coreMPE.src.adenzu_panel.image_processing import panel


_ = panel.extract_panels_for_images_in_folder_recursive(
    input_dir=str(settings.paths.pages_dir),
    output_dir=str(settings.paths.panels_dir),
    split_joint_panels=False,   # maps to --split-joint-panels
    fallback=True              # maps to --fallback
)

### Head crops

In [ ]:
from src.preprocess.headExtraction import anime_extraction_recursive


valid_exts = {".jpg", ".jpeg", ".png"}
panel_paths = sorted(
    [p for p in settings.paths.panels_dir.iterdir() if p.suffix.lower() in valid_exts]
)
num_crops = anime_extraction_recursive()

print(f"Extracted {num_crops} faces")

## Catnip core

### MobileNetV2 Model

In [ ]:
from src.recognition.embeddingModel import compute_embeddings, build_model, load_embeddings


# Build fresh model without loading weights
embed_model = build_model(settings.params.img_size, settings.paths.crops_dir, load_weights=False)
compute_embeddings(embed_model, settings.paths.crops_dir, settings.params.img_size)


# Build embedding model (loads saved weights if available)
embed_model = build_model(settings.params.img_size, settings.paths.crops_dir, load_weights=True)
embs, crop_paths = load_embeddings(settings.paths.embs_dir, settings.paths.crops_dir)


In [ ]:
from src.recognition.query import izutsumi_query, izutsuminess_rank


crop, index, score, thre = izutsumi_query(settings.paths.embs_dir,
                                        settings.paths.crops_dir,
                                        settings.params.img_size, 
                                        embed_model, 
                                        izutsumiPaths,
                                        notIzutsumiPaths,
                                        similarity_threshold=-1,
                                        alpha=0.5,
                                        mode='max')

index_log = izutsuminess_rank(settings.paths.embs_dir, settings.paths.crops_dir, embed_model, izutsumiPaths, notIzutsumiPaths)

### YOLOv8 fine-tuning

In [ ]:
from src.training.preparation import prepare_data

prepare_data(izutsumiPaths, notIzutsumiPaths)

In [ ]:
# For Apple Silicon with MPS enabled on izutsumiTraining.yaml
import os
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [ ]:
from src.training.training import train_model, build_model
from ultralytics import YOLO

model = YOLO(settings.paths.model_dir/'yolov8_izutsumi_finetuned.pt')

model.train(
        data="config/izutsumiTraining.yaml",
        epochs=50,
        imgsz=settings.params.img_size,
        batch=16,
        lr0=1e-4,       # lower LR for finetuning
        freeze=9,      # freeze backbone layers
        project="runs/izutsumi_finetune",
        name="exp1",
        device='cpu',
        workers=8,
        resume=False,
        cache=False
    )

In [ ]:
# Evaluate
metrics = model.val()
print(metrics)

# Predict on unseen images
model.predict(
    source="data/recognition/izutsumiTraining/val/images",
    save=True,
    conf=0.5
)

In [ ]:
model.save("data/models/yolov8_izutsumi_finetuned.pt")

## Outputting

In [ ]:
from src.output.output import save_similar_results, char_nearest_neighbor
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15, 5))
plt.bar(range(len(index)), score)
plt.xlabel("Izutsuminess")
plt.ylabel("Ranked Results")
plt.title("Embed score")
plt.show()

In [ ]:
cutoff = 0

results = char_nearest_neighbor(crop, index[cutoff:], score[cutoff:], thre)

In [ ]:
save_similar_results(crop, index, settings.paths.output_dir, score)